# HTTP Request/Response Cycle - Lab

## Introduction 

In this lab, we'll make use of the `requests` module commands and properties seen in the previous lesson, to extract information for a web service called **"Open Notify"** to access NASA's space data. 

## Objectives

You will be able to:

* Understand and explain the HTTP Request/Response cycle
* Make http requests in Python using the ‘requests’ library

## Open Notify 

[Open Notify](http://open-notify.org/)  is an an open source project to provide a simple programming interface for some of NASA’s awesome data. This takes live raw data from NASA's systems and turn them into APIs related to space and spacecraft. We can access following information from open notify. 

* Current Location of the International Space Station

* Overhead Pass Predictions for the International Space Station

* Number of People in Space
    
### API endpoints

OpenNotify has several API endpoints. 
>An endpoint is a server route that is used to retrieve different data from the API. 

For example, the `/comments` endpoint on the Reddit API might retrieve information about comments, whereas the `/users` endpoint might retrieve data about users. To access them, you would add the endpoint to the base url of the API.

For the OpenNotify API, we have following end points 

1. Current Location of the International Space Station `/iss-now.json`
2. Overhead Pass Predictions for the International Space Station `/iss-pass.json`    
3. Number of People in Space `/astros.json`

The json extension simple tells us that the data is being returned in a JSON format.

In this lab, we'll be querying a this API to retrieve live data about the International Space Station (ISS). Details on OpenNofity , endpoints, syntax and services it offers can be viewed [Here](http://open-notify.org/Open-Notify-API/)

![](images/iss.jpg)

### Current location of International Space Station

The first endpoint we'll look at on OpenNotify is the` iss-now.json` endpoint (current location of international space station). This endpoint gets the current latitude and longitude of the International Space Station.  Perform following tasks 
* Make a get request to get the latest position of the international space station from the opennotify api's `iss_now` endpoint at http://api.open-notify.org/iss-now.json
* Check the status code of the response
* Interpret the returned code

In [27]:
# You Code Here
import requests
resp = requests.get('http://api.open-notify.org/iss-now.json')
print(resp.status_code == requests.codes.ok)
resp.status_code

True


200

In [6]:
dict(resp.headers)

{'Server': 'nginx/1.10.3',
 'Date': 'Thu, 06 Jun 2019 17:10:37 GMT',
 'Content-Type': 'application/json',
 'Content-Length': '114',
 'Connection': 'keep-alive',
 'access-control-allow-origin': '*'}

In [2]:
# Your comments 
#Status code is equal to requests codes ok, so the request was successful.

* Print the contents of the response and identify its current location

In [18]:
# You Code Here
import json

#print contents
print(resp.text)

#It is text, so need to convert to dictionary - I'm using a json method.
response = resp.json()
response

{"message": "success", "iss_position": {"longitude": "165.8297", "latitude": "-51.5172"}, "timestamp": 1559841037}


{'message': 'success',
 'iss_position': {'longitude': '165.8297', 'latitude': '-51.5172'},
 'timestamp': 1559841037}

In [19]:
import pandas as pd
response_df = pd.DataFrame(response)
response_df

,message,iss_position,timestamp
latitude,success,-51.5172,1559841037
longitude,success,165.8297,1559841037


In [4]:
# Interpret your results using the API

### Check the next pass of International space station for a given location

Let's repeat the above for the second endpoint `iss_pass.json`. This end point is used to query the next pass of the space station on a given location. Let's just run as above and record your observations.

In [28]:
# You Code Here
resp = requests.get('http://api.open-notify.org/iss-pass.json')
print(resp.status_code == requests.codes.ok)
#response_df = pd.DataFrame(resp.json())
#response_df
resp.status_code

False


400

In [6]:
# Your comments 
# It's no good

So clearly there is something wrong as we had a 400 response. This is how you should always test your responses for validity. 

if we look at the documentation for the OpenNotify API, we see that the ISS Pass endpoint requires two parameters.

> The ISS Pass endpoint returns when the ISS will next pass over a given location on earth. In order to compute this, we need to pass the coordinates of the location to the API. We do this by passing two parameters -- latitude and longitude.

We can do this by adding an optional keyword argument, params, to our request. In this case, there are two parameters we need to pass:

* lat -- The latitude of the location we want.
* lon -- The longitude of the location we want.

Perform the following tasks :
* Set parameters to reflect the lat and long of New York  (40.71, -74)
* Send a get request to OpenNotify passing in the lat long parameters as k:v pairs in a dictionary
* Check the status code and interpret
* Print the header information and the returned content

In [58]:
# You Code Here
# Format of keys came from http://open-notify.org/Open-Notify-API/ISS-Pass-Times/
my_params = {
    "lat": 40.71,
    "lon": -74, 
  }

print(params)

resp = requests.get('http://api.open-notify.org/iss-pass.json', params=my_params)
print(resp.status_code == requests.codes.ok)
print(resp.status_code)

response = resp.json()
response

{'latitude': 40.71, 'longitude': -74}
True
200


{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1559841321,
  'latitude': 40.71,
  'longitude': -74.0,
  'passes': 5},
 'response': [{'duration': 632, 'risetime': 1559842955},
  {'duration': 620, 'risetime': 1559848767},
  {'duration': 556, 'risetime': 1559854650},
  {'duration': 586, 'risetime': 1559860497},
  {'duration': 649, 'risetime': 1559866296}]}

In [59]:
response_df = pd.DataFrame(response['response'])
response_df

,duration,risetime
0,632,1559842955
1,620,1559848767
2,556,1559854650
3,586,1559860497
4,649,1559866296


In [60]:
from datetime import datetime

response_df.risetime = response_df.risetime.map(lambda x: datetime.fromtimestamp(x))
response_df

,duration,risetime
0,632,2019-06-06 13:42:35
1,620,2019-06-06 15:19:27
2,556,2019-06-06 16:57:30
3,586,2019-06-06 18:34:57
4,649,2019-06-06 20:11:36


In [8]:
# Check the API and interpret your results - when will ISS pass over NEW York next ?
# Row 0 - 2019-06-06 at 13:42:35

### Finding the number of people in space

OpenNotify has one more API endpoint, `/astros.json`. It tells you how many people are currently in space. The format of the responses can be studied [HERE](http://open-notify.org/Open-Notify-API/People-In-Space/).

Read the above documentation and perform following tasks:

* Get the response from astros.json endpoint
* Count how many people are currently in space
* List the names of people currently in space.

In [62]:
# You Code Here
#Making request and checking that it was successful
resp = requests.get('http://api.open-notify.org/astros.json')
print(resp.status_code == requests.codes.ok)
resp.status_code

True


200

In [64]:
resp.json()

{'message': 'success',
 'number': 6,
 'people': [{'craft': 'ISS', 'name': 'Oleg Kononenko'},
  {'craft': 'ISS', 'name': 'David Saint-Jacques'},
  {'craft': 'ISS', 'name': 'Anne McClain'},
  {'craft': 'ISS', 'name': 'Alexey Ovchinin'},
  {'craft': 'ISS', 'name': 'Nick Hague'},
  {'craft': 'ISS', 'name': 'Christina Koch'}]}

In [10]:
# Interpret the Results - How many people are in space and what are their names 
# six people - names listed above. They are all on the ISS.

## Summary 

In this lesson we saw how we can use request and response methods to query an Open API. We also saw how to look at the contents returned with the API calls and how to parse them. Next, we'll look at connecting to APIs which are not OPEN, i.e. we would need to pass in some authentication information and filter the results. 